In [ ]:
# ===== Импорты и инициализация =====

!pip install scikit-learn

from grid.model.navigation.objectinspection import ObjectInspect
from grid.model.navigation.visualservoing import VisualServoing
from grid.model.perception.detection.rt_detr import RT_DETR
from grid.model.perception.depth.metric3d import Metric3D

import airgen
import numpy as np
import rerun as rr
import cv2
import time
from math import cos, sin, pi
from sklearn.cluster import DBSCAN

# Инициализация моделей
nav_inspect = ObjectInspect()
nav_visualservoing = VisualServoing()
detection_rt_detr = RT_DETR(use_local=True)  # Инициализация модели детекции

DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/MultiScaleDeformableAttention-1.0-py3.11-linux-x86_64.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/conda/lib/python3.11/site-packages/openseed-0.1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..


In [ ]:
# ===== Подключение к дрону =====

client = airgen.MultirotorClient()
client.confirmConnection()
client.enableApiControl(True)
client.armDisarm(True)
# ===== Взлет дрона =====

#client.takeoffAsync().join()

# ===== Посадка дрона =====

#client.landAsync().join()

Connected!
Client Ver:1 (Min Req: 1), Server Ver:1 (Min Req: 1)



True

In [ ]:
# ===== Просмотр камер привязаных к Alta X =====

scene_client = airgen.VehicleClient()
scene_objects = scene_client.simListSceneObjects()
cameras = [obj for obj in scene_objects if 'camera' in obj.lower()]

print("Available cameras:")
for camera in cameras:
    print(camera)

Available cameras:
PlayerCameraManager_2147482445
CameraManagerActor
ExternalCamera
BP_PIPCamera_C_2147482405
BP_PIPCamera_C_2147482393
BP_PIPCamera_C_2147482381
BP_PIPCamera_C_2147482369
BP_PIPCamera_C_2147482357


In [ ]:
# ===== Получение и логирование GPS =====

gps_data = client.getGpsData()
geo = gps_data.gnss.geo_point
gps_position = np.array([[geo.latitude, geo.longitude, geo.altitude]], dtype=np.float32)
rr.log("drone/gps", rr.Points3D(gps_position))

In [ ]:
# ===== Получение и логирование изображения с камеры =====

image_data = client.getImages("front_center", [airgen.ImageType.Scene])[0]
rgb_image, _ = image_data  # Получаем изображение и позу
rr.log("drone/camera", rr.Image(rgb_image))

In [ ]:
# ===== Работа ИИ камеры на дроне =====

# ===== Инициализация =====
depth_model = Metric3D()
depth_map = depth_model.run(rgb_image)

result = detection_rt_detr.run(input=rgb_image.copy(), confidence_threshold=0.3)

if isinstance(result, dict) and 'boxes' in result:
    filtered_boxes = []
    filtered_labels = []
    filtered_distances = []

    ignore_labels = ["grass", "floor", "ground", "road", "terrain"]
    min_distance = 2
    max_distance = 20

    # Центр изображения — точка откуда "исходит БПЛА"
    img_center = (rgb_image.shape[1] // 2, rgb_image.shape[0] // 2)

    image_with_lines = rgb_image.copy()

    for box, label in zip(result["boxes"], result["labels"]):
        x, y, w, h = map(int, box)
        cx = x + w // 2
        cy = y + h // 2

        if 0 <= cy < depth_map.shape[0] and 0 <= cx < depth_map.shape[1]:
            distance = depth_map[cy, cx]

            if (min_distance <= distance <= max_distance) and (label.lower() not in ignore_labels):
                filtered_boxes.append([x, y, w, h])
                filtered_labels.append(label)
                filtered_distances.append(distance)

                # ===== Отрисовка линии до объекта =====
                line_color = (0, 255, 0) if distance > 10 else (0, 255, 255) if distance > 5 else (0, 0, 255)
                cv2.line(image_with_lines, img_center, (cx, cy), line_color, 2)

                # Подпись расстояния у центра объекта
                cv2.putText(image_with_lines, f"{distance:.1f}m", (cx + 5, cy),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, line_color, 2)

    # ===== Отрисовка боксов =====
    for i, box in enumerate(filtered_boxes):
        x, y, w, h = box
        cv2.rectangle(image_with_lines, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(image_with_lines, filtered_labels[i], (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        rr.log(
            f"drone/object_{i}",
            rr.Boxes2D(np.array([[x, y, w, h]]), labels=[f"{filtered_labels[i]} - {filtered_distances[i]:.1f}m"])
        )

    # ===== Лог финального изображения =====
    rr.log("drone/camera", rr.Image(image_with_lines))

else:
    print("❗ Объекты не обнаружены.")

❗ Объекты не обнаружены.


In [ ]:
# ===== Получение и подготовка данных LiDAR + Построение маршрута =====

lidar_data = client.getLidarData()
points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
rr.log('lidar/points', rr.Points3D(points))

# ===== Фильтрация: исключаем "землю" (высота < 0.2 м) =====
above_ground = points[points[:, 2] > 0.2]

object_centers = []
object_distances = []

if above_ground.shape[0] > 0:
    # ===== Кластеризация (DBSCAN) =====
    clustering = DBSCAN(eps=1.0, min_samples=10).fit(above_ground[:, :2])
    labels = clustering.labels_

    # ===== Обработка кластеров =====
    unique_labels = set(labels)
    for label in unique_labels:
        if label == -1:
            continue  # -1 означает шум

        cluster_points = above_ground[labels == label]
        center = cluster_points.mean(axis=0)
        distance = np.linalg.norm(center[:2])
        object_centers.append(center)
        object_distances.append(distance)

    rr.log("lidar/objects", rr.Points3D(np.array(object_centers)))
    print(f"🔎 Обнаружено объектов по LiDAR: {len(object_centers)}")
    for i, dist in enumerate(object_distances):
        print(f" - Объект {i+1}: ~{dist:.1f} м")
else:
    print("⚠️ Attention! Дрон на земле или Lidar не нашел вокруг себя объекты")

# ===== Построение кругового маршрута =====
start = client.getMultirotorState().kinematics_estimated.position
takeoff_alt = -15  # вверх в NED
center = start
radius = 10
steps = 12

trajectory = []
for i in range(steps):
    angle = 2 * pi * (i / steps)
    x = center.x_val + radius * cos(angle)
    y = center.y_val + radius * sin(angle)
    z = takeoff_alt
    trajectory.append(airgen.Vector3r(x, y, z))

trajectory.append(airgen.Vector3r(center.x_val, center.y_val, takeoff_alt))  # возврат

rr.log("telemetry/path", rr.LineStrips3D([[(p.x_val, p.y_val, p.z_val) for p in trajectory]]))

⚠️ Attention! Дрон на земле или Lidar не нашел вокруг себя объекты


In [ ]:
print("🟢 Запуск тестового полета Alta X")

# Функция для определения наличия препятствий рядом с дроном
def is_obstacle_near(current_pos, obstacles, threshold=2.0):
    for obstacle in obstacles:
        dist = np.linalg.norm(np.array([current_pos.x_val, current_pos.y_val]) - obstacle[:2])
        if dist < threshold:
            return True, dist
    return False, None

# Функция для поиска безопасного направления
def find_safe_direction(current_pos, obstacles, radius=5.0, angle_step=15):
    best_point = None
    min_density = float('inf')

    for angle in range(0, 360, angle_step):
        rad = np.deg2rad(angle)
        test_x = current_pos.x_val + radius * cos(rad)
        test_y = current_pos.y_val + radius * sin(rad)

        density = 0
        for obs in obstacles:
            dist = np.linalg.norm(np.array([test_x, test_y]) - obs[:2])
            if dist < 2.0:
                density += 1

        if density < min_density:
            min_density = density
            best_point = airgen.Vector3r(test_x, test_y, current_pos.z_val)

    return best_point

# Функция для расчёта угла поворота между точками
def calculate_heading(p1, p2):
    dx = p2.x_val - p1.x_val
    dy = p2.y_val - p1.y_val
    return np.degrees(np.arctan2(dy, dx))

# ===== Основной маршрут =====

# Сохраняем начальное положение и ориентацию
initial_pos = client.getMultirotorState().kinematics_estimated.position
initial_heading = client.getMultirotorState().kinematics_estimated.orientation

for i, target_point in enumerate(trajectory):
    while True:
        current_pos = client.getMultirotorState().kinematics_estimated.position
        lidar_data = client.getLidarData()
        points = np.array(lidar_data.point_cloud, dtype=np.float32).reshape(-1, 3)
        rr.log('lidar/points', rr.Points3D(points))

        above_ground = points[points[:, 2] > 0.2]

        if above_ground.shape[0] == 0:
            print("⚠️ Attention! Дрон на земле или Lidar не нашел вокруг себя объекты")
            obstacle_centers = np.empty((0, 3))
        else:
            clustering = DBSCAN(eps=1.0, min_samples=10).fit(above_ground[:, :2])
            labels = clustering.labels_

            obstacle_centers = []
            for label in set(labels):
                if label == -1:
                    continue
                cluster_points = above_ground[labels == label]
                center = cluster_points.mean(axis=0)
                obstacle_centers.append(center)

            obstacle_centers = np.array(obstacle_centers)

        rr.log("lidar/obstacles", rr.Points3D(obstacle_centers))

        danger, dist = is_obstacle_near(current_pos, obstacle_centers)
        if danger:
            print(f"⚠️ Attention! Угроза на расстоянии {dist:.1f} м. Поиск безопасного пространства...")
            safe_point = find_safe_direction(current_pos, obstacle_centers)
            if safe_point:
                heading = calculate_heading(current_pos, safe_point)
                print("🔵 Безопасное пространство найдено, маршрут перестроен... ")
                client.rotateToYawAsync(heading, 2).join()  # Уменьшена скорость поворота в 2 раза
                client.moveToPositionAsync(safe_point.x_val, safe_point.y_val, safe_point.z_val, 2.0).join()  # Увеличена скорость движения
                print("🟣 Достигнуто безопасное место, движение по изначальной траектории...")
                continue
        break

    heading = calculate_heading(current_pos, target_point)
    client.rotateToYawAsync(heading, 2).join()  # Уменьшена скорость поворота в 2 раза
    client.moveToPositionAsync(target_point.x_val, target_point.y_val, target_point.z_val, 2.0).join()  # Увеличена скорость движения

# ======= Завершающая посадка =======

print("🔶 Alta X находится над назначеной точкой, посадка...")

final_pos = client.getMultirotorState().kinematics_estimated.position
points_raw = client.getLidarData().point_cloud

if len(points_raw) > 0:
    points = np.array(points_raw, dtype=np.float32).reshape(-1, 3)
    ground_points = points[points[:, 2] < 0.5]
    if len(ground_points) > 0:
        min_z = float(np.min(ground_points[:, 2]) - 0.1)
    else:
        min_z = -3.0
else:
    min_z = -3.0

# Плавное снижение к земле
client.moveToPositionAsync(final_pos.x_val, final_pos.y_val, min_z, 0.8).join()

client.landAsync().join()
print("🟢 Дрон осуществил посадку, тестовый полет окнчен.")

🟢 Запуск тестового полета Alta X
⚠️ Attention! Дрон на земле или Lidar не нашел вокруг себя объекты


🔶 Alta X находится над назначеной точкой, посадка...


🟢 Дрон осуществил посадку, тестовый полет окнчен.


In [ ]:
# ==== Движение БПЛА: шаблон для тестов алгоритмов ====

# Повернуть на определённый угол (по оси Yaw, в градусах)
# Положительное значение — поворот по часовой, отрицательное — против часовой

# ==== Угл поворота ==== 

#yaw_angle = 45  # <- угл
#client.rotateByYawRateAsync(yaw_rate=yaw_angle, duration=2).join() # <- инициализация

# ==== Движение вперед ==== 

#forward_distance = 3  # <- дальность в метрах
#client.moveByVelocityAsync(vx=1.0, vy=0.0, vz=0.0, duration=forward_distance).join() # <- инициализация

# ==== Лево - право ==== 

# Движение влево (vy > 0) или вправо (vy < 0)
#side_distance = 3  # <- дальность в метрах 
#client.moveByVelocityAsync(vx=0.0, vy=1.0, vz=0.0, duration=side_distance).join()  # <- инициализация

# Примечание:
# - vx — движение вперёд/назад (м/с), vy — влево/вправо, vz — вверх/вниз
# - duration — время движения в секундах